# Réalisez un dashboard
## Notebook - Test de l'API  
OpenClassrooms - Parcours Data Scientist - Projet 08  

__Ce notebook teste l'API en ligne :__  
* Sélection d'un exemple dans le jeu de test  
* Interrogation de l'API  
* Affichage de la probabilité de défaut de remboursement
* Affichage de l'accord ou non du crédit
* Affichage des 20 éléments ayant eu le plus d'impact sur la prédiction

In [1]:
import pandas as pd
import numpy as np
import requests
import json
import os

In [2]:
# Nombre d'exemples à tester
n_samples = 1

In [3]:
# Récupération des données de test
base_dir = os.getcwd()
df_test_path = os.path.join(base_dir, '..', '..', 'P08 - Dashboard', 'data', 'raw', 'application_test.csv')
df_test = pd.read_csv(df_test_path)

In [4]:
# Récupération du seuil de classification
threshold_path = os.path.join(base_dir, '..', 'data', 'processed', 'best_threshold.txt')
with open(threshold_path, 'r') as threshold_file:
    best_threshold = float(threshold_file.read())

In [5]:
# Sélection aléatoire de n_samples exemples
df_subset = df_test.sample(n_samples)

In [6]:
# Récupération des id clients
sk_id_curr = df_subset['SK_ID_CURR'].astype('int')

In [7]:
# Suppression des id clients du subset
df_subset = df_subset.drop(columns='SK_ID_CURR')

In [8]:
# Remplacement des valeurs manquantes par None (JSON n'accepte pas les NaN)
df_subset =  df_subset.map(lambda x: None if pd.isna(x) else x)

In [9]:
# Conversion au format JSON
data_json = json.dumps({
    "columns": df_subset.columns.tolist(),
    "data": df_subset.values.tolist()}
)

In [25]:
# Envoi de la requête POST à l'API
heroku_url = 'https://failurescore-bc9f53f25e58.herokuapp.com/predict'
local_url = 'http://127.0.0.1:5000/predict'
response = requests.post(
    heroku_url,
    headers={'Content-Type': 'application/json'},
    data=data_json
)

In [26]:
# Vérification du code status
display(response)

<Response [200]>

In [27]:
# DataFrame pour affichage des probabilités et accord ou non de crédit
predictions = pd.DataFrame({'Customer id': sk_id_curr.values.tolist(), 'Failure probability': response.json()['prediction_proba'], 'Credit agreement': response.json()['prediction_class']})
predictions['Credit agreement'] = predictions['Credit agreement'] == 0
predictions['Failure probability'] = round(predictions['Failure probability'], 2)

In [28]:
# Affichage
display(predictions)

,Customer id,Failure probability,Credit agreement
0,233783,0.83,False


In [29]:
# DataFrame pour affichage Feature importance locale
local_fi = pd.DataFrame({'Feature': response.json()['feature_names'], 'Importance': response.json()['feature_importance'][0]})

In [30]:
# Affichage des 20 features ayant eu le plus d'impact sur la prédiction
local_fi.sort_values(by='Importance', key=np.abs, ascending=False).head(20)

,Feature,Importance
30,EXT_SOURCE_3,1.154449
244,CREDIT_TERM,0.398335
29,EXT_SOURCE_2,0.112790
7,AMT_GOODS_PRICE,0.098031
28,EXT_SOURCE_1,0.068190
77,DEF_60_CNT_SOCIAL_CIRCLE,0.060514
240,YEARS_BIRTH,-0.055377
11,OWN_CAR_AGE,0.055043
123,NAME_EDUCATION_TYPE_Highereducation,0.049574
10,DAYS_ID_PUBLISH,0.048980
